## Import libraries

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import GenerationConfig

/opt/conda/envs/rl_algos/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Load Model

In [3]:
model_name = "Qwen/Qwen3-0.6B"

# load the tokenizer and the model
tokenizer = AutoTokenizer.from_pretrained(model_name)
ref_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="bfloat16",
    device_map="cuda:0"
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="bfloat16",
    device_map="cuda:1"
)

In [4]:
# ref_model

## Load dataset

In [4]:
from datasets import load_dataset
def get_dataset(split="train"):
    dataset = load_dataset("openai/gsm8k", "main", split=split)
    QAs = [{'Q':x, 'A':y.split('####')[-1].strip()} for x,y in zip(dataset['question'], dataset['answer'])]
    return QAs

train_dataset = get_dataset("train")
test_dataset = get_dataset("test")

## Constants

In [5]:
GBS = 32
MBS=4
SEQ_LEN=256
GROUPS_SIZE=8


NUM_GEN=8
MAX_PROMPT_LENGTH=512
generation_config = GenerationConfig(
            max_new_tokens=512,
            do_sample=True, 
            temperature=0.9, 
            num_return_sequences=NUM_GEN,
            pad_token_id=tokenizer.pad_token_id,
        )

system_prompt = """You are a helpful assistant. A conversation between User and Assistant. The user asks a question, and the Assistant solves it. The Assistant first thinks about the reasoning process in the mind and then provides the user with the answer.\
The reasoning process and answer are enclosed within <think> </think> and<answer> </answer> tags, respectively, i.e., <think> reasoning process here </think><answer> answer here </answer>."""




## Evaluation script

In [6]:
test_dataset[0]["Q"]

"Janet’s ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market?"

In [8]:
tmp = tokenizer.apply_chat_template([
                {"role":"system", "content":system_prompt},
                {"role":"user", "content":test_dataset[0]["Q"]},]
                                          ,tokenize=False
                                          ,add_generation_prompt=True)

In [11]:
len(tokenizer.tokenize(tmp, return_tensors="pt"))

163

In [ ]:
def generate_answers(dataset):
    with torch.inference_mode():
        inputs=[]
        for question,answer in dataset:
            inputs.append(tokenizer.apply_chat_template([
                {"role":"system", "context":system_prompt},
                {"role":"user", "context":question},]
                                          ,tokenize=False
                                          ,add_generation_prompt=True))

            input_tokens = 